<a href="https://colab.research.google.com/github/GustaveRw/MINECOFIN-Feedback-Analysis/blob/master/MINECOFIN_Feedback_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape
# !pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 

import numpy as np 

import snscrape.modules.twitter as sntwitter 

import datetime 

from tqdm.notebook import tqdm_notebook 

import seaborn as sns 

import matplotlib.pyplot as plt 

sns.set_theme(style="whitegrid") 

In [ ]:
text = input('Enter query text to be matched (or leave it blank by pressing enter)') 

username = input('Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): ') 

since = input('Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 

until = input('Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 

count = (input('Enter max number of tweets or enter -1 to retrieve all possible tweets: ')) 

retweet = input('Exclude Retweets? (y/n): ') 

replies = input('Exclude Replies? (y/n): ')

Enter query text to be matched (or leave it blank by pressing enter)
Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): 
Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): 
Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): 
Enter max number of tweets or enter -1 to retrieve all possible tweets: 
Exclude Retweets? (y/n): 
Exclude Replies? (y/n): 


In [ ]:
import datetime

def search(text, username='', since='', until='', retweet='n', replies='n'):
    global filename
    
    # Convert tuple arguments to strings
    if isinstance(text, tuple):
        text = ' '.join(text)
    if isinstance(username, tuple):
        username = ' '.join(username)
    if isinstance(since, tuple):
        since = ' '.join(since)
    if isinstance(until, tuple):
        until = ' '.join(until)
    
    q = text

    if username:
        q += f' from:{username}'

    if not since:
        since = (datetime.date.today() - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    q += f' since:{since}'

    if not until:
        until = datetime.date.today().strftime('%Y-%m-%d')
    q += f' until:{until}'

    if retweet == 'n':
        q += ' exclude:retweets'

    if replies == 'n':
        q += ' exclude:replies'

    # Generate filename based on search parameters
    if username and text:
        filename = f'{since}_{until}_{username}_{text}.csv'
    elif username:
        filename = f'{since}_{until}_{username}.csv'
    else:
        filename = f'{since}_{until}_{text}.csv'

    print(filename)
    return q


In [ ]:
from tqdm.notebook import tqdm_notebook

q = search(text, username, since, until, retweet, replies)

# Creating list to append tweet data
tweets_list1 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
if count == -1:
    for i, tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q).get_items())):
        tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username, tweet.lang, tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount, tweet.media])
else:
    with tqdm_notebook(total=count) as pbar:
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
            if i >= count:
                break
            tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username, tweet.lang, tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount, tweet.media])
            pbar.update(1)

# Creating a DataFrame from the tweets list above
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username', 'Language', 'Hashtags', 'ReplyCount', 'RetweetCount', 'LikeCount', 'QuoteCount', 'Media'])

# Dropping duplicates based on tweet ID
tweets_df1.drop_duplicates(subset='TweetId', keep='first', inplace=True)

# Resetting index of the DataFrame
tweets_df1.reset_index(drop=True, inplace=True)

# Converting 'DateTime' column to datetime format
tweets_df1['DateTime'] = pd.to_datetime(tweets_df1['DateTime'])

# Setting 'TweetId' column as the index
tweets_df1.set_index('TweetId', inplace=True)

# Sorting the DataFrame by 'DateTime' column
tweets_df1.sort_values(by='DateTime', inplace=True)

# Printing the first 5 rows of the DataFrame
print(tweets_df1.head())


2023-02-22_2023-03-01_.csv


0it [00:00, ?it/s]

ERROR:snscrape.base:Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=+since%3A2023-02-22+until%3A2023-03-01&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&include_ext_ed

ScraperException: ignored